#### <h1>Capstone week 4 assignment Roy</h1> 

We will analyse different boroughs in Oslo Norway and check correlations between depression against factors such as number of foursquare venues, income and education levels. 


*Depresjonsplager hos unge i ulike bydeler

korrelasjon med foursquare venue count  
korrelasjon med foursquare inntekt 
korrelasjon med foursquare utdanning  


First import some required libraries

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Setup locations for each borough. Could not find any online datasets for it.

In [37]:
boroughs_array = np.array([
    ["Alna","59.935015","10.881657"],
    ["Bjerke","59.937422","10.821668"],
    ["Frogner","59.916131","10.706230"],
    ["Gamle Oslo","59.910543","10.765616"],  
    ["Grorud","59.957500","10.880833"],    
    ["Grünerløkka","59.924934","10.770492"],    
    ["Nordre Aker","59.9572374","10.76628"],
    ["Nordstrand","59.861111","10.800278"],
    ["Sagene","59.937222","10.756111"],
    ["St. Hanshaugen","59.927106","10.741061"],
    ["Stovner","59.961111","10.926667"],
    ["Søndre Nordstrand","59.832958","10.819148"],
    ["Ullern","59.928551","10.662471"],
    ["Vestre Aker","59.943003","10.658020"],
    ["Østensjø","59.888889","10.833611"]   
])
boroughs_array.shape

(15, 3)

In [50]:
boroughs = pd.DataFrame({'Borough': boroughs_array[:, 0], 'Latitude': boroughs_array[:, 1], 'Longitude': boroughs_array[:, 2]})
boroughs["Latitude"] = pd.to_numeric(boroughs["Latitude"])
boroughs["Longitude"] = pd.to_numeric(boroughs["Longitude"])
boroughs.set_index('Borough')
boroughs

,Borough,Latitude,Longitude
0,Alna,59.935015,10.881657
1,Bjerke,59.937422,10.821668
2,Frogner,59.916131,10.706230
3,Gamle Oslo,59.910543,10.765616
4,Grorud,59.957500,10.880833
5,Grünerløkka,59.924934,10.770492
6,Nordre Aker,59.957237,10.766280
7,Nordstrand,59.861111,10.800278
8,Sagene,59.937222,10.756111
9,St. Hanshaugen,59.927106,10.741061


Show boroughs on a map

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [39]:
# create map of Oslo 
map_oslo = folium.Map(location=[59.937222,10.756111], zoom_start=12)

# add markers to map
for lat, lng, borough in zip(boroughs['Latitude'], boroughs['Longitude'], boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_oslo)  
    

map_oslo

In [43]:
education_path="http://statistikkbanken.oslo.kommune.no/webview/velocity?headers=Utdanning&Geografisubset=30100%2C30101+-+30118&stubs=Geografi&Utdanningslice=10&measure=common&virtualslice=Antall_value&layers=r&layers=Kjnn&layers=Alder&layers=virtual&study=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfStudy%2FBefokningen-etter-alder-kjonn-utdanning-2017&Aldersubset=7&Alderslice=7&mode=cube&Geografislice=30100&virtualsubset=Antall_value&v=2&rsubset=2018&measuretype=4&Kjnnsubset=3&rslice=2018&cube=http%3A%2F%2Fstatistikkbanken.oslo.kommune.no%3A80%2Fobj%2FfCube%2FBefokningen-etter-alder-kjonn-utdanning-2017_C1&Utdanningsubset=10%2C4&Kjnnslice=3&top=yes&executespreadsheet=true"
education = pd.read_excel(education_path,skiprows=11,names=["Borough","TotalEducation","TopEducation"])


In [69]:
education.replace({'Bydel ':''}, regex=True,inplace=True)
education['Borough'] = education['Borough'].str.strip()
educationShare = education
educationShare['PercentTopEducation'] = (educationShare['TopEducation'] / educationShare['TotalEducation']) *100
educationShare.set_index('Borough')
educationShare

,Borough,TotalEducation,TopEducation,PercentTopEducation
0,Gamle Oslo,47023,10154,0.215937
1,Grünerløkka,52740,12098,0.229389
2,Sagene,38035,10186,0.267806
3,St. Hanshaugen,34059,9418,0.276520
4,Frogner,52624,14278,0.271321
5,Ullern,27203,7954,0.292394
6,Vestre Aker,38529,11321,0.293831
7,Nordre Aker,41190,11589,0.281355
8,Bjerke,25394,3770,0.148460
9,Grorud,22379,1940,0.086688


In [45]:
income_path="http://statistikkbanken.oslo.kommune.no/webview/velocity?headers=innt_num&stubs=geografi&innt_numslice=1&measure=common&geografislice=30101&virtualslice=brutto_sum_value&layers=kjonn_num&layers=virtual&layers=aar&aarslice=2017&study=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfStudy%2Finntektintervall-ny&kjonn_numslice=1&mode=cube&innt_numsubset=1%2C18&virtualsubset=brutto_sum_value&v=2&aar=2017&kjonn_numsubset=1&aarsubset=2007+-+2017&geografisubset=30101+-+30115&measuretype=4&cube=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfCube%2Finntektintervall-ny_C1&top=yes&executespreadsheet=true"
income = pd.read_excel(income_path,skiprows=11,names=["Borough","TotalIncome","TopIncome"])

In [70]:
income.replace({'Bydel ':''}, regex=True,inplace=True)
income['Borough'] = income['Borough'].str.strip()
incomeShare = income
incomeShare['PercentTopIncome'] = (incomeShare['TopIncome'] / incomeShare['TotalIncome']) *100
incomeShare.set_index('Borough')
incomeShare


,Borough,TotalIncome,TopIncome,PercentTopIncome
0,Sagene,20390,3162,0.155076
1,St Hanshaugen,17231,3271,0.189832
2,Frogner,25006,7045,0.281732
3,Ullern,13629,5152,0.378017
4,Vestre Aker,19514,7404,0.379420
5,Nordre Aker,20681,6481,0.313379
6,Bjerke,14293,1767,0.123627
7,Grorud,12006,802,0.066800
8,Stovner,13306,1014,0.076206
9,Alna,21108,1500,0.071063


In [88]:
depression_path="http://statistikkbanken.oslo.kommune.no/webview/velocity?headers=r&virtualsubset=Andel_value&v=2&stubs=Region&measure=common&virtualslice=Andel_value&Regionsubset=0301%2C030101a+-+030115a&rsubset=2015&layers=virtual&measuretype=4&study=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfStudy%2FDepresjonsplager%21hos%21unge%21etter%21bydel&cube=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfCube%2FDepresjonsplager%21hos%21unge%21etter%21bydel_C1&mode=cube&top=yes&executespreadsheet=true"
depressionShare = pd.read_excel(depression_path,skiprows=8,names=["Borough","PercentDepression"])


,Borough,PercentDepression
0,Gamle Oslo,18.0
1,Grunerløkka,18.8
2,Sagene,21.2
3,St. Hanshaugen,15.9
4,Frogner,19.0
5,Ullern,15.2
6,Vestre Aker,13.8
7,Nordre Aker,15.7
8,Bjerke,17.0
9,Grorud,16.8


In [ ]:
depressionShare['Borough'] = depressionShare['Borough'].str.strip()
depressionShare['PercentDepression'] = 100-depressionShare['PercentDepression']
depressionShare.set_index('Borough')
depressionShare

Read income and education data

Merge datasets

In [71]:
incomeAndEducation = pd.merge(incomeShare, educationShare, on='Borough')
incomeAndEducation.drop(columns=['TotalIncome', 'TotalEducation','TopEducation','TopIncome'],inplace=True)
incomeAndEducation.set_index('Borough')
incomeAndEducation

,Borough,PercentTopIncome,PercentTopEducation
0,Sagene,0.155076,0.267806
1,Frogner,0.281732,0.271321
2,Ullern,0.378017,0.292394
3,Vestre Aker,0.379420,0.293831
4,Nordre Aker,0.313379,0.281355
5,Bjerke,0.123627,0.148460
6,Grorud,0.066800,0.086688
7,Stovner,0.076206,0.066235
8,Alna,0.071063,0.092526
9,Østensjø,0.153766,0.149853


In [89]:
incomeEducationDepression =  pd.merge(incomeAndEducation, depressionShare, on='Borough') 
incomeEducationDepression

,Borough,PercentTopIncome,PercentTopEducation,PercentDepression
0,Sagene,0.155076,0.267806,21.2
1,Frogner,0.281732,0.271321,19.0
2,Ullern,0.378017,0.292394,15.2
3,Vestre Aker,0.379420,0.293831,13.8
4,Nordre Aker,0.313379,0.281355,15.7
5,Bjerke,0.123627,0.148460,17.0
6,Grorud,0.066800,0.086688,16.8
7,Stovner,0.076206,0.066235,15.3
8,Alna,0.071063,0.092526,17.4
9,Østensjø,0.153766,0.149853,15.9


In [78]:
boroughsCombined = incomeAndEducation.join(boroughs.set_index('Borough'), on='Borough')
boroughsCombined

,Borough,PercentTopIncome,PercentTopEducation,Latitude,Longitude
0,Sagene,0.155076,0.267806,59.937222,10.756111
1,Frogner,0.281732,0.271321,59.916131,10.706230
2,Ullern,0.378017,0.292394,59.928551,10.662471
3,Vestre Aker,0.379420,0.293831,59.943003,10.658020
4,Nordre Aker,0.313379,0.281355,59.957237,10.766280
5,Bjerke,0.123627,0.148460,59.937422,10.821668
6,Grorud,0.066800,0.086688,59.957500,10.880833
7,Stovner,0.076206,0.066235,59.961111,10.926667
8,Alna,0.071063,0.092526,59.935015,10.881657
9,Østensjø,0.153766,0.149853,59.888889,10.833611


In [38]:
places.shape

(103, 3)

Read lat long data for postal areas

In [39]:


http://statistikkbanken.oslo.kommune.no/webview/velocity?headers=r&virtualsubset=Andel_value&v=2&stubs=Region&measure=common&virtualslice=Andel_value&Regionsubset=0301%2C030101a+-+030115a&rsubset=2015&layers=virtual&measuretype=4&study=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfStudy%2FDepresjonsplager%21hos%21unge%21etter%21bydel&cube=http%3A%2F%2F192.168.101.44%3A80%2Fobj%2FfCube%2FDepresjonsplager%21hos%21unge%21etter%21bydel_C1&mode=cube&top=yes&executespreadsheet=true

Merge the data with our boroughs dataset and fix the Neighbourhood column name so it can be merged again later on column key with correct name

In [73]:
# import k-means from clustering stage
from sklearn.cluster import KMeans


Let's visualizat Manhattan the neighborhoods in it.

In [74]:
# The code was removed by Watson Studio for sharing.

## Explore Neighborhoods in Oslo

In [75]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Create a function to repeat the same process to all the boroughs in Oslo

In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
venues = getNearbyVenues(names=boroughsCombined['Borough'],
                                   latitudes=boroughsCombined['Latitude'],
                                   longitudes=boroughsCombined['Longitude']
                                  )

Sagene
Frogner
Ullern
Vestre Aker
Nordre Aker
Bjerke
Grorud
Stovner
Alna
Østensjø
Nordstrand
Søndre Nordstrand


Check the size of the resulting dataframe

In [84]:
print(venues.shape)
#venues.head()
venues

(137, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sagene,59.937222,10.756111,Mysterud Bar,59.937574,10.755505,Bar
1,Sagene,59.937222,10.756111,Sagene Torg,59.937919,10.754910,Deli / Bodega
2,Sagene,59.937222,10.756111,Klatreverket,59.935141,10.759219,Climbing Gym
3,Sagene,59.937222,10.756111,Myrens Sportssenter,59.934832,10.759601,Gym
4,Sagene,59.937222,10.756111,Kaffe Gram,59.937677,10.751595,Coffee Shop
5,Sagene,59.937222,10.756111,Sagene Lunsjbar,59.937497,10.756574,Breakfast Spot
6,Sagene,59.937222,10.756111,Myraløkka,59.936583,10.757550,Park
7,Sagene,59.937222,10.756111,Bamboo,59.936892,10.752142,Sushi Restaurant
8,Sagene,59.937222,10.756111,Pizza Pancetta,59.939261,10.758959,Pizza Place
9,Sagene,59.937222,10.756111,SATS,59.936277,10.753304,Gym / Fitness Center


Check how many venues were returned for each neighborhood

In [85]:
venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Alna,6,6,6,6,6,6
Bjerke,4,4,4,4,4,4
Frogner,43,43,43,43,43,43
Grorud,8,8,8,8,8,8
Nordre Aker,5,5,5,5,5,5
Nordstrand,16,16,16,16,16,16
Sagene,35,35,35,35,35,35
Stovner,4,4,4,4,4,4
Søndre Nordstrand,3,3,3,3,3,3


Check how many unique categories can be curated from all the returned venues

In [86]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 62 uniques categories.


## Analyze Each Neighborhood

In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
toronto_onehot.shape

(1720, 240)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,...,0.000000,0.00000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.035714,0.000000,0.059524,0.011905,0.000000,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.012048,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.000000,0.00000,0.00,0.000000,0.000000,0.012048,0.000000,0.000000,0.012048,0.00


Check the new size

In [53]:
toronto_grouped.shape

(39, 240)

#### Print each neighborhood along with the top 5 most common venues

In [54]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1  Thai Restaurant  0.04
2              Bar  0.04
3             Café  0.04
4       Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3                Café  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3              Bakery  0.05
4  Italian Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0      Comic Shop  0.07
1   Auto Workshop  0.07
2            Park  0.07
3      Restaurant  0.07
4  Farmers Market  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0

#### Put grouped venues into a *pandas* dataframe

Function to sort the venues in descending order.

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Burger Joint,Steakhouse,Bakery,Sushi Restaurant,Cosmetics Shop,Restaurant
1,Berczy Park,Coffee Shop,Farmers Market,Cocktail Bar,Café,French Restaurant,Cheese Shop,Seafood Restaurant,Bakery,Steakhouse,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Furniture / Home Store,Italian Restaurant,Stadium,Bar,Intersection,Bakery,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Auto Workshop,Comic Shop,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Spa,Farmers Market
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [57]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Pub,Health Food Store,Other Great Outdoors,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Dessert Shop,Diner,Pub,Caribbean Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pizza Place,Burrito Place,Sushi Restaurant,Fish & Chips Shop,Brewery,Ice Cream Shop,Pub,Movie Theater,Italian Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Gastropub,Italian Restaurant,Brewery,Bakery,Bar,Stationery Store,Fish Market
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Photography Studio,Park,Construction & Landscaping,Swim School,Bus Line,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Visualize the resulting clusters

In [59]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[43.8, -79.3], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Dessert Shop,Diner,Pub,Caribbean Restaurant
42,East Toronto,0,Sandwich Place,Pizza Place,Burrito Place,Sushi Restaurant,Fish & Chips Shop,Brewery,Ice Cream Shop,Pub,Movie Theater,Italian Restaurant
43,East Toronto,0,Café,Coffee Shop,American Restaurant,Gastropub,Italian Restaurant,Brewery,Bakery,Bar,Stationery Store,Fish Market
45,Central Toronto,0,Department Store,Park,Food & Drink Shop,Hotel,Sandwich Place,Breakfast Spot,Gym,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
46,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Spa,Shoe Store,Salon / Barbershop,Café,Restaurant,Chinese Restaurant,Pet Store
47,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Café,Gym,Italian Restaurant,Sushi Restaurant,Coffee Shop,Deli / Bodega,Indoor Play Area
49,Central Toronto,0,Coffee Shop,Pub,Light Rail Station,American Restaurant,Restaurant,Liquor Store,Fried Chicken Joint,Sports Bar,Supermarket,Sushi Restaurant
51,Downtown Toronto,0,Coffee Shop,Park,Café,Restaurant,Italian Restaurant,Bakery,Pizza Place,Pub,Gastropub,Diner
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Café,Fast Food Restaurant,Pub,Gym,Hotel
53,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Theater,Café,Breakfast Spot,Restaurant,Mexican Restaurant,Ice Cream Shop


#### Cluster 2

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Playground,Tennis Court,Park,Restaurant,Women's Store,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner,Doner Restaurant
50,Downtown Toronto,1,Park,Trail,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store


#### Cluster 3

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2,Photography Studio,Park,Construction & Landscaping,Swim School,Bus Line,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 4

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Pool,Garden,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 5

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,4,Trail,Pub,Health Food Store,Other Great Outdoors,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
